In [1]:
import tensorflow as tf
import gensim.models.word2vec as word2vec
import multiprocessing
import os

In [2]:
raw_corpus = open('macmorpho-v1-dev.txt').readlines()

In [3]:
raw_corpus[0:2], len(raw_corpus)

(['Por_PREP o_ART conteúdo_N de_PREP cada_PROADJ nova_ADJ lei_N que_PRO-KS-REL entra_V em_PREP vigor_N ,_, nota_V se_PROPESS um_ART aperfeiçoamento_N de_PREP a_ART malha_N jurídica_ADJ em_PREP essa_PROADJ área_N e_KC também_PDEN o_ART afunilamento_N de_PREP o_ART cerco_N fiscal_ADJ em_PREP torno_PREP de_PREP o_ART contribuinte_N ._.\n',
  'Agora_ADV ,_, nenhum_PROADJ cartório_N poderá_VAUX registrar_V quaisquer_PROADJ negócios_N ,_, operações_N ou_KC transações_N de_PREP imóveis_N rurais_ADJ sem_PREP o_ART comprovante_N de_PREP quitação_N de_PREP o_ART ITR_NPROP ._.\n'],
 2211)

### Split words from tokens

In [4]:
tokens = []
unique_tokens = []
corpus = []

for sentence in raw_corpus:
    sentence_tokens = []
    sentence_words = []
    for word in sentence.split():
        sentence_tokens.append(word.split('_')[1])
        unique_tokens.append(word.split('_')[1])
        sentence_words.append(word.split('_')[0])
    tokens.append(sentence_tokens)
    corpus.append(sentence_words)
    
unique_tokens=set(unique_tokens)

In [5]:
unique_tokens

{'!',
 '"',
 '$',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '?',
 'ADJ',
 'ADV',
 'ADV-KS',
 'ADV-KS-REL',
 'ART',
 'CUR',
 'IN',
 'KC',
 'KS',
 'N',
 'NPROP',
 'NUM',
 'PCP',
 'PDEN',
 'PREP',
 'PRO-KS',
 'PRO-KS-REL',
 'PROADJ',
 'PROPESS',
 'PROSUB',
 'V',
 'VAUX',
 '`'}

In [6]:
print(tokens[0:2], len(tokens))

[['PREP', 'ART', 'N', 'PREP', 'PROADJ', 'ADJ', 'N', 'PRO-KS-REL', 'V', 'PREP', 'N', ',', 'V', 'PROPESS', 'ART', 'N', 'PREP', 'ART', 'N', 'ADJ', 'PREP', 'PROADJ', 'N', 'KC', 'PDEN', 'ART', 'N', 'PREP', 'ART', 'N', 'ADJ', 'PREP', 'PREP', 'PREP', 'ART', 'N', '.'], ['ADV', ',', 'PROADJ', 'N', 'VAUX', 'V', 'PROADJ', 'N', ',', 'N', 'KC', 'N', 'PREP', 'N', 'ADJ', 'PREP', 'ART', 'N', 'PREP', 'N', 'PREP', 'ART', 'NPROP', '.']] 2211


In [7]:
print(corpus[0:2], len(corpus))

[['Por', 'o', 'conteúdo', 'de', 'cada', 'nova', 'lei', 'que', 'entra', 'em', 'vigor', ',', 'nota', 'se', 'um', 'aperfeiçoamento', 'de', 'a', 'malha', 'jurídica', 'em', 'essa', 'área', 'e', 'também', 'o', 'afunilamento', 'de', 'o', 'cerco', 'fiscal', 'em', 'torno', 'de', 'o', 'contribuinte', '.'], ['Agora', ',', 'nenhum', 'cartório', 'poderá', 'registrar', 'quaisquer', 'negócios', ',', 'operações', 'ou', 'transações', 'de', 'imóveis', 'rurais', 'sem', 'o', 'comprovante', 'de', 'quitação', 'de', 'o', 'ITR', '.']] 2211


### Max sentence size

In [8]:
max_sentence = ''

for sentence in corpus:
    max_sentence = sentence if len(sentence) > len(max_sentence) else max_sentence

In [9]:
print(max_sentence, len(max_sentence))

['Leitura', 'Formando', 'Crianças', 'Leitoras', 'E', 'Produtoras', 'De', 'Textos', '(', '2', 'VOLS.', ')', ',', 'de', 'J.', 'Jolibert', 'Ler', 'E', 'Dizer', '-', 'Compreensão', 'E', 'Comunicação', 'Do', 'Texto', 'Escrito', ',', 'de', 'Elie', 'Bajard', 'Trabalho', 'O', 'Futuro', 'Do', 'Trabalho', ',', 'de', 'Marcia', 'de', 'Paula', 'Leite', 'Qualidade', 'De', 'Vida', 'NOTRABALHO', ',', 'de', 'Marcus', 'Vinicius', 'Carvalho', 'Rodrigues', 'Contos', 'Os', 'Gatos', 'Nus', 'Passeiam', 'Sobre', 'Os', 'Telhados', 'Sujos', ',', 'de', 'Josette', 'Lassance', 'Depois', 'Do', 'Século', ',', 'de', 'Tina', 'Schumacher', 'Outros', 'Piloto', 'Ianomâmi', ',', 'de', 'Tenn', 'Simioni', 'Condomínio', 'Do', 'Diabo', ',', 'de', 'Alba', 'Zaluar', 'Identidade', 'e', 'fluidez', 'em', 'a', 'sociedade', 'moderna', 'Editoria', ':', 'Mais', 'PáGINA', ':', '6-12', 'Identidade', 'e', 'fluidez', 'em', 'a', 'sociedade', 'moderna', 'Gilberto', 'Velho', 'aplica', 'o', 'conceito', 'de', 'metamorfose', 'a', 'a', 'análise'

## Word2Vec

In [10]:
num_features = 300

In [11]:
min_word_count = 1
num_workers = multiprocessing.cpu_count()
context_size = 7
downsampling = 1e-3

In [12]:
word2vec_model = word2vec.Word2Vec.load(os.path.join("trained", "word2vec.w2v"))

In [13]:
word2vec_model.wv.most_similar("mulher")

[('cabeça', 0.9169880151748657),
 ('mãe', 0.9164813756942749),
 ('filha', 0.9095824956893921),
 ('escola', 0.8843256235122681),
 ('mão', 0.8800747990608215),
 ('figura', 0.8661203384399414),
 ('porta', 0.8607738614082336),
 ('cama', 0.8491104245185852),
 ('frente', 0.8490560054779053),
 ('personalidade', 0.8461546897888184)]

### get a feature vector corpus

In [ ]:
feature_corpus = []

